In [133]:
#путь к данным
path = './data/server3.lpm.org.ru/~baulin/mmCIF/out/'

# Получаем список нужных файлов и цепей из NR-списка

In [134]:
#NR-лист
filter_data = open('./data/nrlist_3.36_all.csv')

filter_outfile = [] 
for i, line in enumerate(filter_data):
    class_ = line.split('\",\"')[1].split('+')
    filter_outfile += class_

filter_data.close()

In [135]:
files_chains = {}
for class_ in filter_outfile:
    class_ = class_.split('|')
    file_name = class_[0].lower() + '.out' + class_[1].strip()
    try:
        file = open(path + file_name)
    except IOError as e:
        continue
    else:
        with file:
            if file_name not in files_chains:
                files_chains[file_name] = []
            files_chains[file_name].append(class_[2])

In [136]:
#смотрим, норм ли напарсили
for i, x in enumerate(files_chains):
    print x, files_chains[x]
    if i>5:
        break

2mfc.out1 ['B']
1zig.out1 ['A']
4k4s.out1 ['B', 'F', 'C', 'G', 'D', 'H']
5dno.out1 ['B']
5x3z.out1 ['B']
3cjz.out1 ['A', 'B']
1arj.out7 ['N']


# Строим набор компонент связности по файлу

In [137]:
import re
stem_re  = r'List of (\d+) stems'
stack_re = r'List of (\d+) stacks'
neighbor_re = r'Summary of structural features of (\d+) nucleotides'

def get_matrix(data, chains):

    #Достаем ребра из блока 'List of (\d+) stems'
    m_stem = re.search(stem_re, data)
    data_stems = ''
    stems = []
    if m_stem is not None:
        data_stems = data[m_stem.end():].split('****************************************************************************')[0]
        stems = data_stems.split('--------------------------------------------------------------------------')
    stems_output = []
    for stem in stems:
        stack1 = []
        stack2 = []
        for line in stem.split('\n'):
            line = line.split()
            if len(line) > 0:
                if line[0].isdigit():
                    stack1.append(line[1])
                    stack2.append(line[2])
        for i in range(len(stack1)-1):
            stems_output.append({'edge':(stack1[i], stack1[i+1]), 'type':'stem'} )
        for i in range(len(stack2)-1):
            stems_output.append({'edge':(stack2[i], stack2[i+1]), 'type':'stem'})


    #Достаем ребра из блока 'List of (\d+) stacks'
    m_stack = re.search(stack_re, data)
    data_stacks=''
    if m_stack is not None:
        data_stacks = data[m_stack.end():].split('****************************************************************************')[0]
    stacks_output = []
    for stack in data_stacks.split('\n'):
        stack = stack.strip().split()
        if len(stack)>0:
            if stack[0].isdigit():
                stack = stack[3].split(',')
                for i in range(len(stack)-1):
                    stacks_output.append({'edge':(stack[i], stack[i+1]), 'type':'stack'} )


    #Достаем ребра из блока 'Summary of structural features of (\d+) nucleotides'
    m_neighbor = re.search(neighbor_re, data) 
    data_neighbors=''
    if m_neighbor is not None:
        data_neighbors = data[m_neighbor.end():].split('****************************************************************************')[0]
    neighbor_output = []
    last_item = None
    for stack in data_neighbors.split('\n'):
        stack = stack.strip().split()
        if len(stack)>0:
            if stack[0].isdigit():
                if last_item is None:
                    last_item = stack[3]
                else:
                    if last_item.split('.')[:-3]==stack[3].split('.')[:-3]:
                        neighbor_output.append({'edge':( last_item, stack[3]), 'type':'neighbor'} )
                        last_item = stack[3]
                    else:
                        last_item = stack[3]


    #Все ребра между нуклеотидами (без связей внутри А-минора)
    edges = stacks_output+stems_output + neighbor_output


    #Достаем А-миноры 'List of (\d+) A-minor motifs'
    a_minor_re = r'List of (\d+) A-minor motifs'
    m_a_minor = re.search(a_minor_re, data) 
    a_minor_data=''
    
    if m_a_minor is not None:
        a_minor_data = data[m_a_minor.end():].split('****************************************************************************')[0]

            
            
#собираем граф        

    a_minors = {}
    a_minor_vert = set()
    last_item = None
    for stack in a_minor_data.split('\n'):
        stack = stack.strip().split()
        if len(stack)>0:
            if stack[0].isdigit():
                if stack[3].split('.')[2] in chains:
                    a_minor_vert.add(stack[3])
                f_stack, mid_stack = stack[3].split('|')
                s_stack, th_stack  = mid_stack.split(',')
                if f_stack not in a_minors:
                    a_minors[f_stack] = []
                a_minors[f_stack].append(stack[3])
                if s_stack not in a_minors:
                    a_minors[s_stack] = []
                a_minors[s_stack].append(stack[3])
                if th_stack not in a_minors:
                    a_minors[th_stack] = []
                a_minors[th_stack].append(stack[3])

    #Матрица смежности А-миноров

    matrix = {}
    matrix_type = {}
    for i, x in enumerate(a_minor_vert):
        matrix[x]={}
        matrix_type[x] = {}
        for y in a_minor_vert:
            if x<y:
                matrix[x][y]=[]
                matrix_type[x][y]=[]
                

    edges = stacks_output+stems_output + neighbor_output 


    for e in edges:
        a, b = min(e['edge']), max(e['edge'])
        if a in a_minors and b in a_minors:
            for x in a_minors[a]:
                if x in a_minor_vert:
                    for y in a_minors[b]:
                        if y in a_minor_vert:
                            if x < y:
                                matrix[x][y].append(e['type']+'_'+a+'_'+b)
                            if x > y:
                                matrix[y][x].append(e['type']+'_'+a+'_'+b)


    for x in matrix:
        matrix[x][x]=[]                                
                                
#связь между А-минорами по общим нуклеотидам
    for c in a_minors:
        if len(a_minors[c])>1:
            for i in range(len(a_minors[c])):
                if a_minors[c][i] in a_minor_vert:
                    for j in range(len(a_minors[c])):
                        if a_minors[c][j] in a_minor_vert:
                            if a_minors[c][i]<a_minors[c][j]:
                                matrix[a_minors[c][i]][a_minors[c][j]].append('a_minor_'+c)
                            else: 
                                matrix[a_minors[c][j]][a_minors[c][i]].append('a_minor_'+c)

    for x in matrix:
        for y in matrix:
            if x < y:
                matrix[y][x]=matrix[x][y]            
        matrix[x][x]=[]
    
    for x in matrix:
        for y in matrix:
            for t in matrix[x][y]:
                if t.find('a_minor_')!= -1:
                    matrix[y][x] = []
                    matrix[x][y] = []
    
    
    for x in matrix_type:
        for y in matrix_type:
            if x<y:
                eat = {}
                for e in matrix[x][y]:
                    et, e1, e2 = e.split('_')
                    ax, nnx = x.split('|')
                    lnx, rnx  = nnx.split(',')
                    ay, nny = y.split('|')
                    lny, rny  = nny.split(',')

                    type_str = ''
                    if lnx in (e1, e2):
                        type_str += 'L'
                    elif rnx in (e1, e2):
                        type_str += 'R'
                    elif ax in (e1, e2):
                        type_str += 'A'
                    if lny in (e1, e2):
                        type_str += 'L'
                    elif rny in (e1, e2):
                        type_str += 'R'
                    elif ay in (e1, e2):
                        type_str += 'A'

                    if type_str not in eat:
                        eat[type_str] = []
                    eat[type_str].append(et[0])
                    eat[type_str].sort()
                for e in eat:
                    new_t = ''.join(eat[e])+e
                    if x<y and new_t not in matrix_type[x][y]:
                        matrix_type[x][y].append(new_t)
                        matrix_type[x][y].sort()
                    if y<x:
                        matrix_type[y][x].append(new_t)
                        matrix_type[y][x].sort()
                        
                        
    #заполняем симметрично с учетом того, что изменилось "направление" ребер 
    for x in matrix_type:
        for y in matrix_type:
            if x < y:
                matrix_type[y][x]=[item[:-2]+item[-1]+ item[-2] for item in matrix_type[x][y]]
        matrix_type[x][x]=[]
        
        
    

    return matrix, matrix_type, len(a_minor_vert)


# Связность компонент

In [153]:
#наличие цикла в данной компоненте
has_cycle = 0
#порядок обхода
order = 0

def sub_dfs(u, matrix, visited, component):
    global has_cycle
    global order
    visited[u][0] = 1
    visited[u][2] = order
    order += 1
    numerator = {}
    
    for v in matrix[u]:
        if len(matrix[u][v])>0:
            a, b = min((u, v)), max((u, v))
            if {'edge': (a, b), 'links': matrix[a][b], 'edge_pattern':visited[a][1] + '_'+visited[b][1]} not in component:
                if visited[v][0] == 1:
                    has_cycle+=1
                component.append({'edge': (a, b), 'links': matrix[a][b], 'edge_pattern': visited[a][1] + '_'+visited[b][1]})
            if visited[v][0] == 0:
                sub_dfs(v, matrix, visited, component)
    visited[u][0] = 2
    

def dfs(matrix):
    global has_cycle
    global order
    text=[]
    cycle_text = []
    components = []
    line_components = []
    cycle_components = []
    visited = {ver:[0, 'v'+str(i), 0] for i, ver in enumerate(matrix)}
    component = []
    for v in matrix:            
        if not visited[v][0]:
            component = []
            has_cycle = 0
            sub_dfs(v, matrix, visited, component)
            if len(component) > 0:
                components.append(component)
            if has_cycle > 0:
                cycle_components.append(component)
            elif len(component)>1:
                line_components.append(component)
    order = 0
    has_cycle = 0
    return components, cycle_components, line_components
    
    

# Вытаскиваем ветки

In [154]:
def get_line(comp):
    matrix = {}
    number_vert={}
    for i, e in enumerate(comp):
        v1, v2 = e['edge_pattern'].split('_')
        
        if v1 not in number_vert:
            number_vert[v1]=0
        number_vert[v1]+=1
        if v2 not in number_vert:
            number_vert[v2]=0
        number_vert[v2]+=1
        if v1 not in matrix:
            matrix[v1]={}
        if v2 not in matrix:
            matrix[v2]={}
        matrix[v1][v2] = [1]
        matrix[v2][v1] = [1]
    visited = {ver:[0, ver, 0] for i, ver in enumerate(matrix)}
    for v in number_vert:
        if number_vert[v]==1:
            if visited[v][0]==0:
                component = []
                sub_dfs(v, matrix, visited, component)
                for x in comp:
                    for i, y in enumerate(component):
                        x1, x2 = x['edge_pattern'].split('_')
                        y1, y2 = y['edge_pattern'].split('_')
                        #if visited[min(x1, x2)] > visited[max(x1, x2)]
                            
                
                        if min(x1, x2)==min(y1, y2) and max(x1, x2)==max(y1, y2):
                            #component[i]['edge']=
                            component[i]['edge']=x['edge']
                            component[i]['links']=x['links']
                            component[i]['type_pattern']=x['type_pattern']
                return component

In [155]:
get_line([{'edge_pattern': '8_7', 'edge': ('1..C.A.46.|1..C.U.92.,1..C.A.114.', '1..C.A.93.|1..C.C.44.,1..C.G.161.'), 'links': ['stack_1..C.A.46._1..C.A.93.', 'neighbor_1..C.A.93._1..C.U.92.'], 'type_pattern': ['nLA', 'sAA', 'nAL']},
{'edge_pattern': '8_10', 'edge': ('1..C.A.46.|1..C.U.92.,1..C.A.114.', '1..C.A.47.|1..C.C.91.,1..C.G.115.'), 'links': ['stack_1..C.A.46._1..C.A.47.', 'stem_1..C.C.91._1..C.U.92.', 'stem_1..C.A.114._1..C.G.115.', 'neighbor_1..C.A.46._1..C.A.47.', 'neighbor_1..C.C.91._1..C.U.92.', 'neighbor_1..C.A.114._1..C.G.115.'], 'type_pattern': ['nsAA', 'nsLL', 'nsRR']},
{'edge_pattern': '7_11', 'edge': ('1..C.A.93.|1..C.C.44.,1..C.G.161.', '1..C.A.94.|1..C.U.43.,1..C.G.162.'), 'links': ['stack_1..C.A.93._1..C.A.94.', 'stem_1..C.C.44._1..C.U.43.', 'stem_1..C.G.161._1..C.G.162.', 'neighbor_1..C.C.44._1..C.U.43.', 'neighbor_1..C.A.93._1..C.A.94.', 'neighbor_1..C.G.161._1..C.G.162.'], 'type_pattern': ['nsAA', 'nsLL', 'nsRR']}])

[{'edge': ('1..C.A.93.|1..C.C.44.,1..C.G.161.',
   '1..C.A.94.|1..C.U.43.,1..C.G.162.'),
  'edge_pattern': '11_7',
  'links': ['stack_1..C.A.93._1..C.A.94.',
   'stem_1..C.C.44._1..C.U.43.',
   'stem_1..C.G.161._1..C.G.162.',
   'neighbor_1..C.C.44._1..C.U.43.',
   'neighbor_1..C.A.93._1..C.A.94.',
   'neighbor_1..C.G.161._1..C.G.162.'],
  'type_pattern': ['nsAA', 'nsLL', 'nsRR']},
 {'edge': ('1..C.A.46.|1..C.U.92.,1..C.A.114.',
   '1..C.A.93.|1..C.C.44.,1..C.G.161.'),
  'edge_pattern': '7_8',
  'links': ['stack_1..C.A.46._1..C.A.93.', 'neighbor_1..C.A.93._1..C.U.92.'],
  'type_pattern': ['nLA', 'sAA', 'nAL']},
 {'edge': ('1..C.A.46.|1..C.U.92.,1..C.A.114.',
   '1..C.A.47.|1..C.C.91.,1..C.G.115.'),
  'edge_pattern': '10_8',
  'links': ['stack_1..C.A.46._1..C.A.47.',
   'stem_1..C.C.91._1..C.U.92.',
   'stem_1..C.A.114._1..C.G.115.',
   'neighbor_1..C.A.46._1..C.A.47.',
   'neighbor_1..C.C.91._1..C.U.92.',
   'neighbor_1..C.A.114._1..C.G.115.'],
  'type_pattern': ['nsAA', 'nsLL', 'nsRR'

# Сбор данных

In [157]:
from datetime import datetime

import os

result_path = './result_'+ str(datetime.now()) + '/'
directory = os.path.dirname(result_path)

try:
    os.stat(directory)
except:
    os.mkdir(directory)      
    
count_all = 0
count_cycle = 0
count_a_minor = 0
count_line = 0
f = open(result_path + 'unusual_line', 'w')

for i, x in enumerate(files_chains):
    
    
    #print i, x
    text = []
    cycle_text = []
    line_text = []

    my_matrix = get_matrix(open(path + x).read(), files_chains[x])
    components, cycle_components, line_components = dfs(my_matrix[0])
    
    count_a_minor += my_matrix[2]
    for c in components:
        for i, e in enumerate(c):
            e['type_pattern'] = [t for t in set(my_matrix[1][e['edge'][0]][e['edge'][1]])]
            c[i]=e
        text.append(str(c))
    for c in cycle_components:
        for i, e in enumerate(c):
            e['type_pattern'] = [t for t in set(my_matrix[1][e['edge'][0]][e['edge'][1]])]
            c[i]=e
        cycle_text.append(str(c))
    for c in line_components:
        for i, e in enumerate(c):
            e['type_pattern'] = [t for t in set(my_matrix[1][e['edge'][0]][e['edge'][1]])]
            c[i]=e
            for t in e['type_pattern']:
                if t[-1]!=t[-2]:
                    f.write(str(e['type_pattern']) + '\t' + x.split('.')[0] + '_line\n')
        line_text.append(str(get_line(c)))
    if len(cycle_components)>0:
        count_cycle += len(cycle_components)
        f_comp = open(result_path + x.split('.')[0] + '_cycle', 'w')
        f_comp.write('\n'.join(cycle_text))
        f_comp.close()
    if len(components)>0:
        count_all += len(components)
        f_comp = open(result_path + x.split('.')[0] + '_all', 'w')
        f_comp.write('\n'.join(text))
        f_comp.close()
    if len(line_components)>0:
        count_line += len(line_components)
        f_comp = open(result_path+x.split('.')[0] + '_line', 'w')
        f_comp.write('\n'.join(line_text))
        f_comp.close()
f.close()

In [158]:
print count_all, count_cycle, count_a_minor, count_line

1491 91 6930 445
